In [2]:
import napari
import skimage as sk
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [3]:
im = sk.io.imread('./domes/Dome103_raw.tif')
im = im[1:, :,:] # remove the first image due to its blurriness

In [4]:
# fig, ax = plt.subplot_mosaic([
#     ['1', '2']
# ], figsize=(30,20))


mask = np.empty((0,im.shape[1], im.shape[2])) # empty np array that has 0 Z , 2048 Y , 2048 X
c = 1
for i in im: # Iterate over each YX plane of the image
    i = i.astype(np.uint8)
    gaussian = gaussian_filter(i, sigma=1)
    median = sk.filters.rank.median(gaussian, sk.morphology.star(10))
    # median = median.astype(np.uint8)
    th3 = cv.adaptiveThreshold(median, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 55, -1) # Needs uint8 to make the asaptive threshold mask
    #ax['1'].imshow(median)
    #ax['2'].imshow(th3)
    mask = np.append(mask, np.array(th3))
    # break
    print(c, end="\r")
    c+=1
    


In [5]:
skeleton = np.empty((0,im.shape[1], im.shape[2])) # empty np array that has 0 Z , 2048 Y , 2048 X
mask = np.reshape(mask, (160, 2048, 2048))
for i in mask:
    i = sk.morphology.skeletonize(i)
    skeleton = np.append(skeleton, np.array(i))
skeleton = np.reshape(skeleton, (160, 2048, 2048))

In [7]:
viewer = napari.Viewer()
#viewer.add_image(skeleton)
viewer.add_image(mask, scale=np.array([1,0.1625,0.1625]))
#viewer.add_image(im)

<Image layer 'mask' at 0x2cfd201c890>